In [ ]:
#In this notebook we implement the methods used to evaluate the linear
#relaxations of sparce pca. This notebook corresponds to section 4.2 
#of the paper.

<h1>functions and evaluation of SDP for sparse PCA  and its linear relaxation<h1>

<h2>Table of contents <h2>   
    0. load sparsePCArelaxations notebook, set seed.<br>
    1. Code for figure 2<br>
    2. Code for figure 3<br>
    3. Code for figure 4<br>
    4. Code for table 6<br>
    5. Table for table 7<br>
    

<h2> 0. load sparsePCArelaxations notebook.<h2>

In [ ]:
using NBInclude, LightGraphs,Statistics,Random,Distributions,Plots
 

In [ ]:
@nbinclude("sparsePCArelaxations.ipynb")


In [ ]:
Random.seed!(123412341234)

<h2>1. Code  for table 2 <h2>
    

In [ ]:
# we evaluate the time taken to solve each of the semidefinite program
# and the linear relaxation of SParse PCA. For both programs, we 
#we report the time taken by the solver. For the linear relaxation.

<h3> Functions <h3>

In [ ]:
#function to compute the time taken to solve the semidefinite relaxation of sparse PCA
# in average. We report the standard deviation and the average time.
#@param startSize: size of the first graphs we try.
#@param endSize: largest size of graphs that we try.
#@param sampleSize: number of graph to average
function plotTimesSParsePCASemidefinite(startSize,endSize,sampleSize)
sizeGraph= collect(startSize:5:endSize) 
lenSizeGraph = length(sizeGraph)
results = zeros(3,lenSizeGraph)
results[1,:] = sizeGraph
q=1
for i in sizeGraph
    println(i)
    graphDeg = Int64(round(i/10))
    if isodd(i*graphDeg)
        graphDeg=graphDeg+1
    end
        
    meanTime = zeros(sampleSize)  
        
    for j in  1:sampleSize  
      B=  rand(Normal(0,100),i,i)
                   
    PsdMatrix = (1/(i-1))*B*transpose(B)
    k =  Int64(round(sqrt(i)))
    sparseSolution = SdpSparsePCA(PsdMatrix,k)
    meanTime[j] = sparseSolution[4]
        end
    results[2,q]=  mean(meanTime)
    results[3,q]=  std(meanTime)
    q=q+1
    
    end
   
        
        
    return(results)
end

#function to compute the time taken to solve the linear relaxation of the semidefinite relaxation of sparse PCA.
#@param startSize: size of the first graphs we try.
#@param endSize: largest size of graphs that we try.
#@param sampleSize: number of graph to average
function plotTimesSParsePCAEigenRel(startSize,endSize,sampleSize)
sizeGraph= collect(startSize:5:endSize) 
lenSizeGraph = length(sizeGraph)
results = zeros(3,lenSizeGraph)
results[1,:] = sizeGraph
q=1
 for i in sizeGraph
    print(i)
    graphDeg = Int64(round(i/10))
    if isodd(i*graphDeg)
        graphDeg=graphDeg+1
    end
   meanTime = zeros(sampleSize)  
    for j in  1:sampleSize  
      B=  rand(Normal(0,100),i,i)
                   
    PsdMatrix = (1/(i-1))*B*transpose(B)
    k =  Int64(round(sqrt(i)))
    sparseSolution = EigenRelSparsePCA(PsdMatrix,k)
    meanTime[j] = sparseSolution[4]
        end

        results[2,q]=  mean(meanTime)
    results[3,q]=  std(meanTime)
    q=q+1
    
    end
    
        
        
    return(results)
end



<h3> Compute values <h3>

In [ ]:
timeExperimentsSDP =  plotTimesSParsePCASemidefinite(10,70,5)
timeExperimentsLP =  plotTimesSParsePCAEigenRel(10,70,5)


<h3>Plot results<h3>

In [ ]:
σs = (timeExperimentsSDP[:,3],timeExperimentsSDP[:,3])
plotComparedResults= plot(timeExperimentsSDP[1,:],timeExperimentsSDP[2,:],marker=(:square, 3, 1.), label =("SDP_time"),legend=:topleft,xaxis ="size of matrix",yaxis="time in seconds" ,size = (700, 500),linestyle=:dash,yerror =(timeExperimentsSDP[3,:],timeExperimentsSDP[3,:]),colour="black"  )
plot!(timeExperimentsLP[1,:],timeExperimentsLP[2,:],marker=(:diamond, 3, 1.), label =("EigenLp_time"),legend=:topleft,xaxis ="size of matrix",yaxis="time in seconds" ,size = (700, 500),linestyle=:dash,yerror =(timeExperimentsLP[3,:],timeExperimentsLP[3,:]),colour="black"  )


<h2>Code for Figure 3<h2>

In [ ]:
#Experiments on comparing the optimal value of the linear relaxation for sparse PCA versus
# the optimal value of the semidenifite relaxation, by increasing number of cuts added to the linear program.


<h3>Functions<h3>

In [ ]:

#function to plot the objective of the linear relaxations as more and more cuts are added to the Linear relaxations. We solve the problem for
#numGraphs graphs and at a targetspacity. We divide the obtained values by the objective value of the SDP solution.
#@param int numGraphs the number of graphs for which we solve the problem
#@param matrixType: type of matrices for comparaison. if matrixType== d>0 is a regular graph of degree d. If matrixType==0
# B has entries distributed as normal(0,20).
#@param int targetSparcity. If targetSparcity==0 then we set k= sqrt(i), where i is the graph size.
#@param int matrix_size the number of rows of the matrices to use. Matrices are assumed to be squared.
#@param int rounds is the number of experiments we are doing. Each round adds 5 cuts to the LPs.
 function ObjectiveVSIteration(numGraphs,matrixType,targetSparcity,matrix_size,rounds)

    maxNumCuts = rounds*5
    results  =zeros(7,rounds)
    sdpResults = zeros(numGraphs)
      graphs = Array{Any,1}(undef,numGraphs)
      graphsStoredCuts = Array{Any,1}(undef,numGraphs)
      graphsStoredCutsEigen =  Array{Any,1}(undef,numGraphs)
      [graphsStoredCuts[r] = zeros(matrix_size) for r in 1:numGraphs]
     
       if matrixType==0
        
            for r in 1: numGraphs
            currentGraph = rand(Normal(0,20),matrix_size,matrix_size)
            graphs[r]=  (1/(matrix_size-1))*currentGraph*transpose(currentGraph)
            end 
       else 
             if isodd(matrixType)
             matrixType=matrixType+1
             end
        
           for r in 1: numGraphs
            currentGraph = Matrix(adjacency_matrix(random_regular_graph(matrix_size,matrixType)))
            graphs[r]=  (1/(graphSize-1))*currentGraph*transpose(currentGraph)
            end 
    
    
    end

       [ graphsStoredCutsEigen[r] = eigen(graphs[r]).vectors for r in 1:numGraphs]
     
        [sdpResults[r] = SdpSparsePCA(graphs[r],targetSparcity)[3] for r in 1:numGraphs ]
       

    count = 1
     cutValue = zeros(numGraphs)
        cutValueEigen = zeros(numGraphs)
    
    for i in 5:5:maxNumCuts
       
        for j in 1:numGraphs
    
            solvedIterated = solverIteratelyLP(graphs[j],5,targetSparcity, graphsStoredCuts[j],0)  
             cutValue[j]=solvedIterated[1]
           
            graphsStoredCuts[j]=  solvedIterated[2]
            
        cutValue[j]= sdpResults[j]/cutValue[j]
             solvedIteratedEigen = solverIteratelyLP(graphs[j],5,targetSparcity, graphsStoredCutsEigen[j],i)
                cutValueEigen[j]=sdpResults[j]/solvedIteratedEigen[1]
                graphsStoredCutsEigen[j]= solvedIteratedEigen[2]

             
        end
         results[1,count] = i
        results[2,count] = median(cutValue)
        results[3,count] = minimum(cutValue)
        results[4,count] = maximum(cutValue)
        results[5,count] = median(cutValueEigen)
        results[6,count] = minimum(cutValueEigen)
        results[7,count] = maximum(cutValueEigen)
            
       count = count+1 
    end

    return(results)
end

In [ ]:
# Function to compute the linear relaxation of the SDP by adding cuts iteratively. If cuts are precomputed
# they are added as a parameter in Hinicial. This function is called by the function ObjectiveVSIteration.

# @param adjMatrix the  matrix to compute the sparse PCA.
# @param numCuts the number of cuts to generate and add to the linear program.
# @param targetSparcity the sparcity of the PCA components.
# @param Hinicial cuts to add to the linear program, in case they were computed on a previous step.
# @param eigens indicate if  we should use eigenvectors or not. If eigens = 0, the method is not using eigenvectors.
# if eigens = i>0, uses eigens from 1 to i.
function solverIteratelyLP(adjMatrix,numCuts,targetSparcity, Hinicial,eigens)

    n = size(adjMatrix)[1]
    m = Model(Mosek.Optimizer)
    set_silent(m)
    resultantH = Hinicial
      @variable(m,X[1:n,1:n],Symmetric ) 
      @constraint(m,tr(X)==1)
      @variable(m,Y[1:n,1:n],Symmetric)  
          @constraint(m,Y.>=X )
           @constraint(m,Y.>=-X )
           @constraint(m,sum(Y)<=targetSparcity)
    
    if(eigens>0 && eigens<=n)
        
        for s in 1:eigens
        @constraint(m, transpose(Hinicial[:,s])*X*Hinicial[:,s]>=0)  
        end
    else
    
        for s in 1:size(Hinicial,2)
        @constraint(m, transpose(Hinicial[:,s])*X*Hinicial[:,s]>=0)  
        end
    end
    
        @objective(m,Max,tr(adjMatrix*X))
         status = optimize!(m)
    
    if(eigens>n)   
        
     for s in 1:numCuts
      maxEigVecto = eigen(value.(X)).vectors[:,1]
      @constraint(m, transpose(maxEigVecto)*X*maxEigVecto>=0)  
      resultantH = hcat(resultantH,maxEigVecto)
      status = optimize!(m)
      end        
        elseif (eigens==0)
            for s in 1:numCuts
      maxEigVecto = eigen(value.(X)).vectors[:,1]
      @constraint(m, transpose(maxEigVecto)*X*maxEigVecto>=0)  
      resultantH = hcat(resultantH,maxEigVecto)
      status = optimize!(m)
      end  
            end
                
        
    return(objective_value(m),resultantH)

end

<h3>Compute values<h3>

In [ ]:
 comparedResults  =  ObjectiveVSIteration(20,0,10,80,30)


<h3>Plot values<h3>

In [ ]:
 styles = filter((s->begin
                s in Plots.supported_styles()
        end), [:solid, :dash, :dash ])
styles = reshape(styles, 1, length(styles))
a = vec(transpose(comparedResults[5,:]) - transpose(comparedResults[6,:]))
b = vec(transpose(comparedResults[7,:])-transpose(comparedResults[5,:]))
plotComparedResults= plot(comparedResults[1,:], comparedResults[5,:],marker=(:square, 3, 1.), label =("EigenCuts/sdp median"),legend=:topleft,xaxis ="cuts",yaxis="Lp/SDP" ,size = (700, 500),linestyle=:dash,yerror =(a,b),colour="black"  )
a2 = vec(transpose(comparedResults[2,:])-transpose(comparedResults[3,:]))
b2 = vec(transpose(comparedResults[4,:])-transpose(comparedResults[2,:]))
plot!(comparedResults[1,:].+1.5, comparedResults[2,:], label =("LinearCuts/sdp median"),legend=:topleft,marker=(:diamond, 3, 1.),xaxis ="number of cuts",yaxis="Lp/SDP" ,size = (700, 500),linestyle =:dash,yerror =(a2,b2),colour="black")
 

<h2>Code for Figure 4 <h2>

In [ ]:
# We plot the time taken to produce cuts and add them iteratively versus 
#finding the eigenvectors of the matrix

<h3>Functions<h3>

In [ ]:
#function to compare the methods of creating cuts vs our eigenRelaxation
#For these experiments, we set the number of generated cuts to n, the size of the graph
# we measure the time taken to generate the n cuts using either the eigenDecomposition
# and the LPs.
# for the matrices, we simply use a matrix with normal (0,20) entries.
# We measure only the time required to solve the optimization programs, and not the time 
# taken by JUMP to formulate them.
function compareTimes(maxGraphSizes,targetSparcity,sampleSize)
    len = length(10:5:maxGraphSizes)
    results = zeros(5,len)
    count = 1
for i in 10:5:maxGraphSizes
        meanTimeLPcuts = zeros(sampleSize)  
        meanTimeEigen =  zeros(sampleSize)
            B = rand(Normal(0,20),i,i)
            currentGraph=  (1/(i-1))*B*transpose(B)
                
       results[1,count]=i
        
        for j in 1:sampleSize
        meanTimeLPcuts[j]= iterativeLptime(currentGraph,targetSparcity)
        meanTimeEigen[j] =     EigenLptime(currentGraph,targetSparcity)
      
        end
         results[2,count]= mean(meanTimeLPcuts)
       results[3,count]= std(meanTimeLPcuts)
        results[4,count] = mean(meanTimeEigen)
          results[5,count] = std(meanTimeEigen)
       count = count+1
    end
        return(results)
end



# measures the time taken to iteratively solve the SDP    
function iterativeLptime(adjMatrix,targetSparcity)
        
         n = size(adjMatrix)[1]
    m = Model(Mosek.Optimizer)
    set_silent(m)
      @variable(m,X[1:n,1:n],Symmetric ) 
      @constraint(m,tr(X)==1)
      @variable(m,Y[1:n,1:n],Symmetric)  
          @constraint(m,Y.>=X )
           @constraint(m,Y.>=-X )
           @constraint(m,sum(Y)<=targetSparcity)
         @objective(m,Max,tr(adjMatrix*X))
         status = optimize!(m)
    timetoSolveLp = solve_time(m)
   for s in 1:n
      maxEigVecto = eigen(value.(X)).vectors[:,1]
      @constraint(m, transpose(maxEigVecto)*X*maxEigVecto>=0)  
      status = optimize!(m)
      timetoSolveLp = timetoSolveLp + solve_time(m) 
            
        end
        return(timetoSolveLp)
    end
            
function EigenLptime(adjMatrix,targetSparcity)
        
         n = size(adjMatrix)[1]
    m = Model(Mosek.Optimizer)
    set_silent(m)
       @variable(m,X[1:n,1:n],Symmetric ) 
      @constraint(m,tr(X)==1)
      @variable(m,Y[1:n,1:n],Symmetric)  
          @constraint(m,Y.>=X )
           @constraint(m,Y.>=-X )
           @constraint(m,sum(Y)<=targetSparcity)
    timeSolveEigen = @elapsed eigeVectors =  eigen(adjMatrix).vectors
        
        
   for s in 1:n
      EigVecto =eigeVectors[:,s]
      @constraint(m, transpose(EigVecto)*X*EigVecto>=0)  
        end
      status = optimize!(m)
      timeSolveEigen = timeSolveEigen + solve_time(m) 
            
        return(timeSolveEigen)
    end
    
    

<h3>Compute values<h3>

In [ ]:
timeResults = compareTimes(100,10,5)


<h3>Plot Results<h3>

In [ ]:
p= plot(timeResults[1,:],timeResults[2,:],marker=(:square, 3, 1.), label =("Linear cuts"),legend=:topleft,xaxis ="size of matrix",yaxis="time in seconds" ,size = (700, 500),linestyle=:dash,yerror =(timeResults[3,:],timeResults[3,:]),colour="black"  )
plot!(timeResults[1,:],timeResults[4,:],marker=(:diamond, 3, 1.), label =("Eigen cuts"),legend=:topleft,xaxis ="size of matrix",yaxis="time in seconds" ,size = (700, 500),linestyle=:dash,yerror =(timeResults[5,:],timeResults[5,:]),colour="black"  )



<h2>4. Code for table 6 <h2>

<h2>Recovery of sparse components: Synthetic experiments.<h2>

In [ ]:
# Recovery of sparse components using the sdp and the linear relaxation on a synthetic experiment.
# This is the same experiment as in "A direct formulation for sparse PCA
#using semidefinite programming" by D'aspermont et al.

In [ ]:
#Define the true covariance matrix 

TrueCov = [290^2 290^2 290^2 290^2 0 0 0 0 -0.3*290^2 -0.3*290^2; 290^2 290^2 290^2 290^2 0 0 0 0 -0.3*290^2 -0.3*290^2;
    290^2 290^2 290^2 290^2 0 0 0 0 -0.3*290^2 -0.3*290^2; 290^2 290^2 290^2 290^2 0 0 0 0 -0.3*290^2 -0.3*290^2; 0 0 0 0 300^2 300^2 300^2 300^2  0.925*300^2 0.925*300^2; 0 0 0 0 300^2 300^2 300^2 300^2  0.925*300^2 0.925*300^2
; 0 0 0 0 300^2 300^2 300^2 300^2  0.925*300^2 0.925*300^2;0 0 0 0 300^2 300^2 300^2 300^2  0.925*300^2 0.925*300^2;  -0.3*290^2 -0.3*290^2 -0.3*290^2 -0.3*290^2 0.925*300^2 0.925*300^2 0.925*300^2 0.925*300^2  (290^2)*(0.3^2)+(300^2)*(0.925)^2 (290^2)*(0.3^2)+(300^2)*(0.925)^2;
-0.3*290^2 -0.3*290^2 -0.3*290^2 -0.3*290^2 0.925*300^2 0.925*300^2 0.925*300^2 0.925*300^2  (290^2)*(0.3^2)+(300^2)*(0.925)^2 (290^2)*(0.3^2)+(300^2)*(0.925)^2]

In [ ]:
# Check the matrix is symmetric.
issymmetric(TrueCov)

<h3> a. Recovery of sparse components using the semidefinite program for sparse PCA<h3>

In [ ]:
SdpSol =SdpSparsePCA(TrueCov,4)[1]
firstPcSdp = eigen(SdpSol).vectors[:,10]


In [ ]:
#Deflate the matrix to obtain next sparse component
deflated = TrueCov-(transpose(firstPcSdp)*TrueCov*firstPcSdp)*firstPcSdp*transpose(firstPcSdp)
sdpSol2 =SdpSparsePCA(deflated,4)[1]
secondPcSdp =eigen(sdpSol2).vectors[:,10]

<h3>b. Recovery of sparse components using the linear relaxation for sparse PCA<h3>


In [ ]:
 linearSol=EigenRelSparsePCA(TrueCov,4)[1]

In [ ]:
 LinearSolFirstPc =eigen(linearSol).vectors[:,10]

In [ ]:
#Deflate the matrix to obtain next sparse component
deflatedCovLinear = TrueCov-(transpose(LinearSolFirstPc)*TrueCov*LinearSolFirstPc)*LinearSolFirstPc*transpose(LinearSolFirstPc)
deflatedCovLinear = round.(deflatedCovLinear,digits=5)
LinearSol2 =EigenRelSparsePCA(deflatedCovLinear,4)[1]
LinearSolSecondPc =eigen(LinearSol2).vectors[:,10]

<h2>5. Code for Table  7<h2>

In [ ]:
#Computation of sparse components using the semidefinite relaxation and the linear relaxation

In [ ]:
#Pitprops dataset

In [ ]:
# load the covariance matrix for the pitpros dataset
data = [1,0.954,0.364,0.342,-0.129,0.313,0.496,0.424,0.592,0.545,0.084,-0.019,0.134,
0.954,1,0.297,0.284,-0.118,0.291,0.503,0.419,0.648,0.569,0.076,-0.036,0.144,
0.364,0.297,1,0.882,-0.148,0.153,-0.029,-0.054,0.125,-0.081,0.162,0.22,0.126,
0.342,0.284,0.882,1,0.22,0.381,0.174,-0.059,0.137,-0.014,0.097,0.169,0.015,
-0.129,-0.118,-0.148,0.22,1,0.364,0.296,0.004,-0.039,0.037,-0.091,-0.145,-0.208,
0.313,0.291,0.153,0.381,0.364,1,0.813,0.09,0.211,0.274,-0.036,0.024,-0.329,
0.496,0.503,-0.029,0.174,0.296,0.813,1,0.372,0.465,0.679,-0.113,-0.232,-0.424,
0.424,0.419,-0.054,-0.059,0.004,0.09,0.372,1,0.482,0.557,0.061,-0.357,-0.202,
0.592,0.648,0.125,0.137,-0.039,0.211,0.465,0.482,1,0.526,0.085,-0.127,-0.076,
0.545,0.569,-0.081,-0.014,0.037,0.274,0.679,0.557,0.526,1,-0.319,-0.368,-0.291,
0.084,0.076,0.162,0.097,-0.091,-0.036,-0.113,0.061,0.085,-0.319,1,0.029,0.007,
-0.019,-0.036,0.22,0.169,-0.145,0.024,-0.232,-0.357,-0.127,-0.368,0.029,1,0.184,
0.134,0.144,0.126,0.015,-0.208,-0.329,-0.424,-0.202,-0.076,-0.291,0.007,0.184,1]
data = reshape(data,13,13)

<h3>a. Recovery of sparse components using the semidefintie program for sparse PCA<h3>

In [ ]:
# First component, k= 5
SdpSolProps=SdpSparsePCA(data,5)[1]
SparseEigenVectorSdpSol =eigen(SdpSolProps).vectors 
firstPCsdp = SparseEigenVectorSdpSol[:,13]

In [ ]:
#second component, k=2
deflate1 = data-(transpose(firstPCsdp)*data*firstPCsdp)*firstPCsdp*transpose(firstPCsdp)
secondPcMatrix = SdpSparsePCA(deflate1,2)[1]
secondPcSdp = eigen(secondPcMatrix).vectors[:,13]

In [ ]:
#Third component, k = 2
deflate2 = deflate1-(transpose(secondPcSdp)*deflate1*secondPcSdp)*secondPcSdp*transpose(secondPcSdp)
thirdPcMatrix = SdpSparsePCA(deflate2,2)[1]
thirdPcSdp = eigen(thirdPcMatrix).vectors[:,13]

<h3>b. Recovery of sparse components using the linear relaxation for sparse PCA<h3>

In [ ]:
#First component, k=5 
linearProp=EigenRelSparsePCA(data,5)[1]
firstLinearPc = eigen(linearProp).vectors[:,13]


In [ ]:
#second component, k=2
deflate1Linear = data-(transpose(firstLinearPc)*data*firstLinearPc)*firstLinearPc*transpose(firstLinearPc)
secondPcMatrixLinear = EigenRelSparsePCA(deflate1Linear,2)[1]
secondPcMatrixLinear = round.(secondPcMatrixLinear,digits=5)
secondPcLinear = eigen(secondPcMatrixLinear).vectors[:,13]

In [ ]:
#Third  component, k = 2
deflate2Linear = deflate1Linear-(transpose(secondPcLinear)*deflate1Linear*secondPcLinear)*secondPcLinear*transpose(secondPcLinear)
ThirdPcMatrixLinear = EigenRelSparsePCA(deflate2Linear,2)[1]
ThirdPcMatrixLinear = round.(ThirdPcMatrixLinear,digits=5)
ThirdPcLinear = eigen(ThirdPcMatrixLinear).vectors[:,13]